In [16]:
using PyCall
using Statistics
using LinearAlgebra
using Random
using Distributions
using TensorCrossInterpolation
import TensorCrossInterpolation as TCI
using JLD2
#using OptimKit
;

In [17]:
# PyCallを用いてPythonのライブラリをインポート
np = pyimport("numpy")
@pyimport tf_quant_finance as tff
@pyimport tensorflow 

In [18]:
# オプション満期
T = 1.0
# 無リスク金利
r = 0.01
# strike
K = 100.0
# 原資産数
d = 5
# 原資産価格初期値
#S0 = [100.0] * d

#l(0.5, d)
# 相関行列


corrMat = Matrix{Float64}(I, d, d)
for i in 1:d
    for j in 1:d
        if i != j
            corrMat[i, j] = 1/2
        end
    end
end



#localindex_v = [51 for _ in 1:d]
#localindex_v  =  [69, 13, 91, 87, 51]
#localindex_v = [79, 66, 54, 76, 62] # const 0.5 
#localindex_v = [74, 86, 70, 54, 68] # const 0.5 with noisy
#localindex_v =  [79, 66, 54, 76, 62] # const 0.5 with noisy
localindex_v = [72, 74, 86, 63, 79]# const 0.5 with noisy
nPath = 50000000
S0 = fill(100, d)
σs = collect(0.15:0.001:0.249)
#σs = collect(0.1:0.003:0.397)

# パス数
#nPath = 10000000
#σs = collect(0.1:0.003:0.397)
#stock0 = collect(80.0:0.5:129.5)
stock0 = collect(90.0:0.3:119.7)

100-element Vector{Float64}:
  90.0
  90.3
  90.6
  90.9
  91.2
  91.5
  91.8
  92.1
  92.4
  92.7
   ⋮
 117.3
 117.6
 117.9
 118.2
 118.5
 118.8
 119.1
 119.4
 119.7

In [19]:
corrMat

5×5 Matrix{Float64}:
 1.0  0.5  0.5  0.5  0.5
 0.5  1.0  0.5  0.5  0.5
 0.5  0.5  1.0  0.5  0.5
 0.5  0.5  0.5  1.0  0.5
 0.5  0.5  0.5  0.5  1.0

In [20]:
#=
corrMat = [ 
    1.0       0.014664  0.10966   0.437896   0.133544 ;
    0.014664  1.0       0.48206   0.585829   0.576213 ;
    0.10966   0.48206   1.0       0.607172   0.524112 ;
    0.437896  0.585829  0.607172  1.0        0.0560393 ;
    0.133544  0.576213  0.524112  0.0560393  1.0]

=#


corrMat = [
    1.0       0.303659  0.505532  0.719655  0.728832;
 0.303659  1.0       0.401077  0.515272  0.178203;
 0.505532  0.401077  1.0       0.132542  0.722767;
 0.719655  0.515272  0.132542  1.0       0.394723;
 0.728832  0.178203  0.722767  0.394723  1.0]
new_order = [1, 4, 5, 3, 2]
reordered_corrMat = corrMat[new_order, new_order]


#=
corrMat = [ 1.0       0.472591  0.595334  0.453276  0.554031;
    0.472591  1.0       0.42697   0.539885  0.533166;
    0.595334  0.42697   1.0       0.531799  0.462572;
    0.453276  0.539885  0.531799  1.0       0.593003;
    0.554031  0.533166  0.462572  0.593003  1.0]
=#


5×5 Matrix{Float64}:
 1.0       0.719655  0.728832  0.505532  0.303659
 0.719655  1.0       0.394723  0.132542  0.515272
 0.728832  0.394723  1.0       0.722767  0.178203
 0.505532  0.132542  0.722767  1.0       0.401077
 0.303659  0.515272  0.178203  0.401077  1.0

In [21]:
is_symmetric(matrix::AbstractMatrix) = matrix == matrix'
is_symmetric(corrMat)

eigvals(corrMat)

5-element Vector{Float64}:
 0.09523938212267367
 0.16366894710092655
 0.8017296544609304
 1.0576484774530504
 2.8817135388624187

In [22]:
len = length(σs)

100

In [23]:
function calculate_option_price(T::Float64, r::Float64, K::Float64, d::Int64, S0::Vector{Int64}, stock0::Vector{Float64}, localindex_v::Vector{Int64}, nPath::Int64, σs::Vector{Float64}, corrMat)::Tuple{Float64, Float64}
    vols = [σs[i] for i in localindex_v[1:d]]
    #S0s = [stock0[i] for i in localindex_v[d+1:2*d]]

    process = tff.models.MultivariateGeometricBrownianMotion(
        dim=d,
        means=fill(r, d),
        #volatilities=fill(0.2, d),
        volatilities=vols,
        corr_matrix=corrMat)
    
    paths = process.sample_paths(
        times=[T],
        #initial_state=S0s,
        initial_state=S0,
        random_type=tff.math.random.RandomType.PSEUDO,
        num_samples=nPath).numpy()
    
    payoffs = max.(minimum(paths[:, 1, :], dims=2) .- K, 0.0)
    #payoffs = vec(payoffs)  # 配列をベクトルに変換
    mean_pv = exp(-r * T) * mean(payoffs)
    std_pv = exp(-r * T) * std(payoffs)
    #pv = exp(-r * T) * mean(max.(minimum(paths[:, 1, :], dims=2) .- K, 0.0))
    return (mean_pv, std_pv)
    #return pv

end

calculate_option_price (generic function with 1 method)

In [24]:
# オプション価格の計算
mean_pv, std_pv = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, reordered_corrMat)
println("Present Value: ", mean_pv)
println("Standard Deviation: ", std_pv)
#pv = calculate_option_price(T, r, K, d, S0, localindex, nPath, σs, corrMat)
#println(pv)
;

Present Value: 1.4412325071590775
Standard Deviation: 4.9101773657159296


In [25]:
true_value = 1.3885828365963908
println("真の値: ", true_value)
;

真の値: 1.3885828365963908


In [26]:
SE = std_pv/sqrt(nPath)
CI = 1.96 * std_pv/sqrt(nPath)
#@show 1.96 * SE
#println("絶対誤差の指標の代わり=", 2 * 1.96 * SE) 
println("CI: ", CI) 
#println("Present Value + CI: ", mean_pv + CI) 
#println("Present Value - CI: ", mean_pv - CI) 
#@show mean_pv + CI #　信頼区間
#@show mean_pv - CI # 信頼区間
;

CI: 0.0013610317271535613


In [27]:
#=
time_mc = @elapsed begin
    for i in 1:20
        calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
    end
end
time_mc/20
=#

In [28]:
function generate_random_combinations(len, d, num_samples)
    Random.seed!(100)

    possible_values = collect(1:len)
    random_combinations = Vector{Int}[]
    
    for _ in 1:num_samples
        combination = rand(possible_values, d)
        push!(random_combinations, combination)
    end
    
    return random_combinations
end


# Option pricing taking random combinations of parameters
function rand_inner(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat, len, num_samples, random_combinations)
    #random_combinations = generate_random_combinations(len, d, num_samples)
    #random_combinations = [[i, 2, 2, 2, 2] for i in 1:len]

    results = Vector{ComplexF64}()
    
    for i in 1:num_samples
        ind = random_combinations[i]
        localindex_v = ind
        #inner_ = inner_p(d, ind, M_phi, M_v)
        #result = (( η^d ) * exp(-r * (T - t)) * inner_) / ((2 * π) ^ d)
        result, _ = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
    
        push!(results, result)
    end
    return results, random_combinations
end

num_samples = 100  # Number of parameter combinations
random_combinations = generate_random_combinations(len, d, num_samples)
results, combinations = rand_inner(T, r, K, d, S0, stock0, localindex_v, nPath, σs, reordered_corrMat, len, num_samples, random_combinations)
mc =  real(results) 
#=
for j in 1:5
    random_combinations = [[i, j+1, 2, 2, 2] for i in 1:len]
    results, combinations = rand_inner(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat, len, num_samples, random_combinations)
    tt = real(results)
    JLD2.@save "vol_mc_d5_50000000_const_diff_$j.jld2" tt
end
=#

100-element Vector{Float64}:
 1.2227466229876292
 1.2713905757099886
 1.2656388191507588
 1.3185051392583156
 1.2838661027430234
 1.322475538686015
 1.2432413853625657
 1.2852258001074515
 1.3360058578319423
 1.2284288403127248
 ⋮
 1.2816793066753813
 1.3453231332752593
 1.3333891449961184
 1.3343804631820848
 1.3536583787910763
 1.1804466665211648
 1.2516818005019366
 1.2142960798495852
 1.2819475361294685

In [29]:
#JLD2.@save "volatility_mc_d5_50000000_rand_extended.jld2" mc
JLD2.@save "volatility_mc_d5_50000000_rand_reordered.jld2" mc